In [79]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb
%run useful_functions.ipynb

In [80]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [81]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide"]

temp_show_broadcast_details = pd.DataFrame()

#for show_url in shows_to_run:
for show_url in shows['Show URL']:
    show_instance = Show(show_url)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True
temp_show_broadcast_details['Is Maybe Pending'] = True

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data','Is Maybe Pending'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [82]:
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & ((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:10]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    if i % 10 == 0:
        print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    if len(broadcast_instance.playlist.index) == 0 and (pd.to_datetime('now')-broadcast_instance.broadcast_details["Service Date"]).dt.days <= 7:
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = True
    else:
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = False
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


Broadcast Number 1 of 10


In [83]:
#show_broadcast_details['Is New Data'] = False
#broadcast_details['Is New Data'] = False
#broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [84]:
mask_2022 = broadcast_details['Service Date'].apply(lambda date: date.year) == 2022

(
broadcast_details[mask_2022]
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key Broadcast Details,Is New Data Broadcast Details,...,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Show Key,Datetime Key Show Details,Is New Data Show Details,Is Maybe Pending,Show URL,Show Name
28,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m0015v4d,Steve recaps the weekend's #6MusicFestival,"#6MusicFestival live tracks from Bloc Party, L...",False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
29,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m0015v4d,Steve recaps the weekend's #6MusicFestival,"#6MusicFestival live tracks from Bloc Party, L...",False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
30,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m0015v4d,Steve recaps the weekend's #6MusicFestival,"#6MusicFestival live tracks from Bloc Party, L...",False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
31,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m0015v4d,Steve recaps the weekend's #6MusicFestival,"#6MusicFestival live tracks from Bloc Party, L...",False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
32,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m0015v4d,Steve recaps the weekend's #6MusicFestival,"#6MusicFestival live tracks from Bloc Party, L...",False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Steve Lamacq,Steve has live music from Bess Atwell,Steve kicks of the week with a Kitchen Disco t...,https://programmes.files.bbci.co.uk/programmes...,Mon 7 Feb 2022,2022-02-07,16:00,-9196240667197234118,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m001475c,Steve has live music from Bess Atwell,And the National Anthem search continues,False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
24,Steve Lamacq,Steve has live music from Bess Atwell,Steve kicks of the week with a Kitchen Disco t...,https://programmes.files.bbci.co.uk/programmes...,Mon 7 Feb 2022,2022-02-07,16:00,-9196240667197234118,2022-12-11 12:37:39.355677,True,...,https://www.bbc.co.uk/programmes/m001475c,Steve has live music from Bess Atwell,And the National Anthem search continues,False,1408665427283283890,2022-12-11 12:37:22.098745,True,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
25,

In [87]:
shows
show_broadcast_details
broadcast_details
broadcast_playlists

,Artist Image URL,Artist Name,Track Name,Record Label,Track Number,Broadcast Position,Broadcast Key,Datetime Key,Is New Data
0,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Jim O’Rourke,Ghost Ship in a Storm,,,1,-9217409020754396024,2022-12-11 12:37:39.356451,True
1,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,ViVii,Suckerpunch,,,2,-9217409020754396024,2022-12-11 12:37:39.356451,True
2,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Here Lies Man,Animal Noises,,,3,-9217409020754396024,2022-12-11 12:37:39.356451,True
3,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Tom Waits,Telephone Call from Istanbul,,,4,-9217409020754396024,2022-12-11 12:37:39.356451,True
4,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Santana,Hope You're Feeling Better,,,5,-9217409020754396024,2022-12-11 12:37:39.356451,True
...,...,...,...,...,...,...,...,...,...
243,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Smog,I Feel Like The Mother Of The World,,,20,-9177229921058442386,2022-12-11 12:37:39.356451,True
244,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Queen,Radio Gaga (Live at Wembley),,,21,-9177229921058442386,2022-12-11 12:37:39.356451,True
245,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Kate Bush,Hello Earth,,,22,-9177229921058442386,2022-12-11 12:37:39.356451,True
246,https://ichef.bbci.co.uk/images/ic/96x96/p01c9...,Sufjan Stevens,America,,,23,-9177229921058442386,2022-12-11 12:37:39.356451,True


In [71]:
broadcast_key

-9177229921058442386